Дз 3. Поиск топиков в текстах, работа с gensim
#### Аня Смирнова БКЛ181 


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import collections

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
#!pip install pyLDAvis

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

выгружаем датасет.
убираем лишние знаки, стоп-слова, пунктуацию

In [8]:
df_raw = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df_raw.target_names.unique())
df_raw.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [9]:
data = df_raw.content.values.tolist()
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


составляем биграмы и триграмы

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


создаем корпус, каждому слову в корпусе соответствует определенный номер (корпус - лист из листов, которые содержат словари айди_слова - частотность)

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


дальше работаем с **mallet**

In [16]:
#!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [18]:
#!unzip mallet-2.0.8.zip

In [19]:
from gensim.models.wrappers import LdaMallet

mallet_path = '/content/mallet-2.0.8/bin/mallet'

функция, которая подбирает оптимальное количество групп топиков (грузится ужасно долго,взяла шаг 10, самое оптимальное вышло **32 группы**)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        mallet_path = '/content/mallet-2.0.8/bin/mallet' 
        ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(ldamallet)
        coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_ldamallet = coherence_model_ldamallet.get_coherence()
        print(coherence_ldamallet)
        coherence_values.append(coherence_ldamallet)

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


0.37658984558561215
0.5035707198498258
0.5371216733238552
0.545619030897206


определяем главный топик в текстах

In [20]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=32, id2word=id2word)
tm_results = ldamallet[corpus] #получаем все топики для всех текстов
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in tm_results] #выбираем самый самый главный топик в каждом тексте


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [21]:
topic_df = pd.DataFrame()
topic_df['Title'] = df_raw.target_names
topic_df['Topic'] = [item[0]+1 for item in corpus_topics]
topic_df['Text'] = texts
topic_df.head()

,Title,Topic,Text
0,rec.autos,13,"[where, thing, car, nntp_poste, host, park, li..."
1,comp.sys.mac.hardware,12,"[poll, final, call, summary, final, call, cloc..."
2,comp.sys.mac.hardware,1,"[engineering, computer, network, distribution_..."
3,comp.graphics,19,"[division, line, host, write, write, article, ..."
4,sci.space,6,"[question, distribution, article, write, clear..."


посчитали в скольких текстах встречаются главные топики

In [22]:
main_df = topic_df.groupby('Topic').agg(
                                  Doc_Count = ('Topic', np.size),
                                  Total_Docs_Perc = ('Topic', np.size)).reset_index()
main_df.head()

,Topic,Doc_Count,Total_Docs_Perc
0,1,121,121
1,2,288,288
2,3,386,386
3,4,988,988
4,5,297,297


считаем **tf-idf**

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm


In [25]:
main_topics = []
for i in topic_df['Topic']:
#print(i)
  main_topics.append(i)
  c = collections.Counter(main_topics)
len(list(c))


  

32

In [44]:
all_groups = [] #список групп
topics = []
for i in list(c):
  group_list = [] #список текстов в группах
  df = topic_df[['Title', 'Topic','Text']]
  df2 = df.groupby(['Topic']).get_group(i)
  for topic in df2['Topic']:
    topics.append(topic)
#print(df2)
  for text in df2['Text']:
    row = ' '.join(text) #это тексты
    #print(row)
    group_list.append(row)
    group = group_list 
  all_groups.append(group)





In [ ]:
scores = []
for group in tqdm(all_groups):
  corpus = []
  for text in group: #берем тексты в группе и добавляем их в корпус
    corpus.append(text)
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(corpus)
    vec = vectors[0]
    df_vec = pd.DataFrame(vec.T.todense(), index=vectorizer.get_feature_names(), columns=["tfidf"])
    words = list(df_vec.sort_values('tfidf', ascending=False).index[:5])
    scores.append(words)
print(scores)

In [51]:
df_final = pd.DataFrame()
df_final['topic'] = topics
df_final['text'] = all_texts
df_final['keywords'] = scores
df_final

,topic,text,keywords
0,13,where thing car nntp_poste host park line wond...,"[car, look, door, see, name]"
1,13,summary s worth distribution_usa may gmt organ...,"[car, door, look, addition, know]"
2,13,insurance rate performance car line latech new...,"[car, door, look, history, bricklin]"
3,13,car saftey important line host write safety im...,"[car, door, look, lerxst, engine]"
4,13,member read internet access system modem line ...,"[car, door, look, history, body]"
...,...,...,...
11309,25,mark line think result last week banner appear...,"[number, cop, estimate, skew, junior]"
11310,25,help line use requirement analsys methodology ...,"[number, gorgeous, density, park, crowd]"
11311,25,superstition line article write mass anectdota...,"[number, gorgeous, density, park, crowd]"
11312,25,line write write regard wilkerson be real sure...,"[number, gorgeous, density, junior, skew]"


coherence score

Благодаря метрикам coherence мы можем разделить условно все топики на плохие и хорошие. Как это происходит:


1.   набор слов t сегментируется на набор пар подмножеств слов S
2.   вероятности слов P вычисляются на основе заданного эталонного корпуса
3.   набор подмножеств слов S и полученные вероятности используются в дальнейшем для получения когерентности C (то есть связи между ними)

По сути этот показатель измеряет топик посредством вычисления степени семантической близости слов с большим весом (то есть большой вероятностью). У этой штуки есть два метода: внутренний (сравнение попарно последовательно идущих слов) и внешний (сравнение любых пар слов в тексте).

